In [8]:
import pandas as pd
import numpy as np
from numpy.random import (
    beta, binomial
)
import scipy.stats as stats
from IPython.display import display
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.stats import beta

In [2]:
Batting = pd.read_csv('data/baseballdatabank-master/core/Batting.csv', index_col=['playerID'])
Pitching = pd.read_csv('data/baseballdatabank-master/core/Pitching.csv', index_col=['playerID'])
Master = pd.read_csv('data/baseballdatabank-master/core/Master.csv', index_col='playerID')

In [3]:
Batting = Batting[~Batting.index.isin(Pitching.index)]
career = (Batting[Batting["AB"] > 0]
            .groupby(level=0)
            .agg({
                'H': np.sum, 
                'AB': np.sum
            })
          .assign(average = lambda r: r['H'] / r['AB']))
career.head()

,H,AB,average
playerID,,,
aaronha01,3771.0,12364.0,0.304998
aaronto01,216.0,944.0,0.228814
abadan01,2.0,21.0,0.095238
abadijo01,11.0,49.0,0.224490
abbated01,772.0,3044.0,0.253614


In [4]:
# priors estimated in chapter 3
alpha0 = 75.88
beta0 = 216.69

career = career.assign(average_estimate=lambda x: (x['H'] + alpha0) / (x['AB'] + alpha0 + beta0))


In [5]:
career = career.assign(alpha1=lambda x: alpha0 + x['H'],
                       beta1=lambda x: beta0 + x['AB'] - x['H'])
career.head()

,H,AB,average,average_estimate,alpha1,beta1
playerID,,,,,,
aaronha01,3771.0,12364.0,0.304998,0.303943,3846.88,8809.69
aaronto01,216.0,944.0,0.228814,0.236040,291.88,944.69
abadan01,2.0,21.0,0.095238,0.248366,77.88,235.69
abadijo01,11.0,49.0,0.224490,0.254355,86.88,254.69
abbated01,772.0,3044.0,0.253614,0.254117,847.88,2488.69


In [7]:
drobs_fav_yankees = [
    'willibe02', 'strawda01','knoblch01',
    'jeterde01', 'posadjo01', 'martiti02',
    'brosisc01']

yankee_1998_career = career.loc[drobs_fav_yankees]
yankee_1998_career

,H,AB,average,average_estimate,alpha1,beta1
playerID,,,,,,
willibe02,2336.0,7869.0,0.296861,0.295517,2411.88,5749.69
strawda01,1401.0,5418.0,0.258583,0.258622,1476.88,4233.69
knoblch01,1839.0,6366.0,0.288878,0.287581,1914.88,4743.69
jeterde01,3465.0,11195.0,0.309513,0.308236,3540.88,7946.69
posadjo01,1664.0,6092.0,0.273145,0.272513,1739.88,4644.69
martiti02,1925.0,7111.0,0.270707,0.270259,2000.88,5402.69
brosisc01,1001.0,3889.0,0.257393,0.257530,1076.88,3104.69


In [12]:
def quantile_of_beta(p, shape1, shape2):
    """
    Calculates the probability density of the Beta distribution
    """
    return beta.ppf(q=p, a=shape1, b=shape2, loc=0, scale=1)

quantile_of_beta(0.91, 2411, 5749)

0.302250630244029